In [1]:
#IMPORT NECESSARY MODULES
import pandas as pd
import datetime
import numpy as np
#CHANGE DISPLAY OPTIONS TO SHOW MORE ROWS AND COLUMNS
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
#LIST OF ALL JOURNALS AND THEIR GROUPS
online_journals = [
    ['American Journal of Distance Education','0892-3647'],
    ['Distance Education','0158-7919'],
    ['E-Learning','1741-8887'],
    ['Electronic Journal of e-Learning','1479-4403'],
    ['International Journal of Distance Education Technologies','1539-3100'],
    ['International Review of Research in Open and Distance Learning','1492-3831'],
    ['International Journal of Mobile and Blended Learning','1941-8647'],
    ['Journal of Educators Online','1547-500X'],
    ['Journal of E-Learning and Knowledge Society','1826-6223'],
    ['Journal of Library and Information Services in Distance Learning','1533-290X'],
    ['Journal of Interactive Online Learning','1541-4914'],
    ['Open Learning','0268-0513'],
    ['Turkish Online Journal of Distance Education','1302-6488'],
]

ed_tech_journals = [
    ['Australasian Journal of Educational Technology','1449-5554'],
    ['British Journal of Educational Technology','0007-1013'],
    ['Bulletin of the Technical Committee on Learning Technology','2306-0212'],
    ['Computer Assisted Language Learning','0958-8221'],
    ['Computers and Education','0360-1315'],
    ['Computers in Education Journal','1069-3769'],
    ['Computers in the Schools','0738-0569'],
    ['Cutting-Edge Technologies in Higher Education','2044-9968'],
    ['DESIDOC Journal of Library and Information Technology','0974-0643'],
    ['Education and Information Technologies','1360-2357'],
    ['Educational Communication and Technology Journal','0148-5806'],
    ['Educational Media International','0952-3987'],
    ['Educational Technology and Society','1436-4522'],
    ['Educational Technology Research and Development','1042-1629'],
    ['Interactive Learning Environments','1049-4820'],
    ['Interactive Technology and Smart Education','1741-5659'],
    ['International Journal of Computer Supported Collaborative Learning','1556-1607'],
    ['International Journal of Educational Technology in Higher Education','EISSN: 2365-9440'],
    ['International Journal of Information and Communication Technology Education','1550-1876'],
    ['International Journal of Information and Learning Technology','2056-4880'],
    ['International Journal of Innovation and Learning','1471-8197'],
    ['International Journal of Learning Technology','1477-8386'],
    ['International Journal of Technologies in Learning','2327-0144'],
    ['International Journal of Technology and Design Education','0957-7572'],
    ['International Journal of Technology Enhanced Learning','1753-5255'],
    ['International Journal of Web-Based Learning and Teaching Technologies','1548-1093'],
    ['Internet and Higher Education','1096-7516'],
    ['Journal of Computer-Assisted Learning','0266-4909'],
    ['Journal of Computing in Higher Education','1042-1726'],
    ['Journal of Educational Computing Research','0735-6331'],
    ['Journal of Educational Multimedia and Hypermedia','1055-8896'],
    ['Journal of Interactive Learning Research','1093-023X'],
    ['Journal of Research on Technology in Education','1539-1523'],
    ['Journal of Technology, Learning, and Assessment','1540-2525'],
    ['Knowledge Management and E-Learning','2073-7904'],
    ['Language, Learning, & Technology','1094-3501'],
    ['Learning, Media and Technology','1743-9884'],
    ['Performance Improvement Quarterly','0898-5952'],
    ['Research in Learning Technology','2156-7069'],
    ['Technology, Knowledge and Learning','2211-1662'],
    ['Technology, Pedagogy and Education','1475-939X'],
    ['TechTrends','8756-3894'],
    ['Turkish Online Journal of Educational Technology','1303-6521']
]
    
education_journals = [
    ['Cognition and Instruction','0737-0008'],
    ['International Journal of Instruction','1694-609X'],
    ['International Journal of Knowledge and Learning','1741-1009'],
    ['International Journal of Learning','1447-9494'],
    ['Instructional Science','0020-4277'],
    ['Journal of Learning Sciences','1050-8406'],
    ['Learning and Instruction','0959-4752'],
    ['Learning Environments Research','1387-1579'],
    ['Memory and Cognition','0090-502X'],
]

computer_science_journals = [
    ['International Journal of Mobile Human Computer Interaction','1942-390X'],
    ['International Journal of Technology and Human Interaction','1548-3908'],
    ['International Journal of Artificial Intelligence in Education','1560-4292'],
    ['Australian Educational Computing','0816-9020'],
    ['Computers in Human Behavior','0747-5632'],
    ['Human-Computer Interaction','0737-0024'],
    ['IEEE Transactions on Learning Technologies','1939-1382'],
    ['Information Technology and Libraries','0730-9295'],
    ['International Journal of Human Computer Studies','1071-5819'],
    ['International Journal of Human-Computer Interaction','1044-7318'],
    ['Journal of Computer-Mediated Communication','1083-6101'],
]
#READ ALL DATA AND APPEND TO NEW DATAFRAME
import os
base = 'C:/Users/bodil/Projects/Rick Journal Data/'
df = pd.DataFrame()
for file_name in os.listdir(base + 'Data/'):
    df2 = pd.read_csv(base + 'Data/' + file_name, low_memory=False)
    df2['journal'] = file_name.split('.')[0]
    df = df.append(df2)
df = df[df.articleType == 'Article'].copy()
df = df[df.abstract == df.abstract].copy()

In [10]:
def parse_abstract(cell):
    cell_list = cell.split('.')
    first_sentence = '©' in cell_list[0] or 'Taylor & Francis' in cell_list [0] or 'All rights reserved' in cell_list[0] or \
    ' LLC.' in cell_list[0] or 'Elsevier' in cell_list[0] or 'Springer' in cell_list[0]
    second_sentence = 'All rights reserved' in cell_list[1]
    last_sentence = '©' in cell_list[-1] or 'Taylor & Francis' in cell_list [-1] or 'All rights reserved' in cell_list[-1] or \
    ' LLC.' in cell_list[-1] or 'Elsevier' in cell_list[-1] or 'Springer' in cell_list[-1]
    if second_sentence and last_sentence:
        return '.'.join(cell_list[2:-1])
    elif second_sentence and not last_sentence:
        return '.'.join(cell_list[2:])
    elif first_sentence and last_sentence:
        return '.'.join(cell_list[1:-1])
    elif first_sentence and not last_sentence:
        return '.'.join(cell_list[1:])
    elif not first_sentence and last_sentence:
        return '.'.join(cell_list[:-1])
    else:
        return '.'.join(cell_list)

In [11]:
df['new_abstract'] = df.abstract.map(parse_abstract)

In [12]:
df.reset_index(inplace=True,drop=True)

In [15]:
from nltk import tokenize
df['sentence_abstract_list'] = df['new_abstract'].map(lambda x: tokenize.sent_tokenize(x))

In [19]:
tempdf = df[['sentence_abstract_list']].copy()

In [23]:
finaldf = pd.DataFrame({
    col:np.repeat(tempdf[col].values, tempdf['sentence_abstract_list'].str.len())
    for col in tempdf.columns.difference(['sentence_abstract_list'])
    }).assign(**{'sentence_abstract_list':np.concatenate(tempdf['sentence_abstract_list'].values)})[tempdf.columns.tolist()]

In [49]:
finaldf['sentence_abstract_series_of_lists_of_words'] = finaldf['sentence_abstract_list'].map(lambda x: x.split(' '))

In [103]:
import string
translator = str.maketrans('','',string.punctuation + '’')
def remove_punctuation(cell):
    word_list = []
    for word in cell:
        word = word.translate(translator)
        if word == ' ' or word == '':
            pass
        else:
            word_list.append(word.lower())
    return word_list

finaldf['sentence_abstract_series_of_lists_of_words'] = finaldf['sentence_abstract_series_of_lists_of_words'].map(remove_punctuation)

In [104]:
sentences = finaldf.sentence_abstract_series_of_lists_of_words.values

In [105]:
print('number of sentences')
print(len(sentences))

number of sentences
334312


In [106]:
from gensim.models import Word2Vec

In [107]:
model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

In [108]:
model.wv.most_similar('learning')

[('srl', 0.5576258897781372),
 ('teachinglearning', 0.5521644949913025),
 ('pbl', 0.5266392827033997),
 ('creativity', 0.5087181925773621),
 ('elearning', 0.4991961121559143),
 ('instruction', 0.4922972619533539),
 ('problemsolving', 0.48335739970207214),
 ('inquiry', 0.4687046706676483),
 ('learner', 0.4595080316066742),
 ('scaffolding', 0.4580767750740051)]

In [111]:
model.wv.most_similar('teaching')

[('instruction', 0.6701321005821228),
 ('pedagogy', 0.6305698156356812),
 ('instructional', 0.5995449423789978),
 ('pedagogical', 0.5675981640815735),
 ('studentcentered', 0.5628814697265625),
 ('classroom', 0.5384673476219177),
 ('curriculum', 0.537446916103363),
 ('teachinglearning', 0.5318211913108826),
 ('classrooms', 0.5195698142051697),
 ('education', 0.5082148313522339)]

In [114]:
#model.wv.most_similar(positive=['technology', 'computer'], negative=['computer'])
model.wv.most_similar(positive=['open'])

[('oer', 0.6091980934143066),
 ('odl', 0.5540707111358643),
 ('internetbased', 0.5339124798774719),
 ('oss', 0.5007176399230957),
 ('ignou', 0.4721396267414093),
 ('australian', 0.4666614532470703),
 ('expanding', 0.46371832489967346),
 ('distance', 0.4586717188358307),
 ('opensource', 0.4582386910915375),
 ('library', 0.4573057293891907)]

In [79]:
model.wv.similarity('engagement', 'self-regulation')

0.56723246672242156

In [126]:
model.wv.doesnt_match("creativity self-regulation motivation teacher".split(' '))

'teacher'

In [80]:
model.score(["The fox jumped over a lazy dog".split()])

RuntimeError: We have currently only implemented score for the hierarchical softmax scheme, so you need to have run word2vec with hs=1 and negative=0 for this to work.